In [1]:
import mlflow

In [2]:
# Define the MLflow storage path on local Drive
mlflow_storage_path = "/Users/lukasfichtner/Documents/Guayas_project_week3/mlflow_results"
# Set MLFlow  to log to zhe local drive directory
mlflow.set_tracking_uri(f"file:{mlflow_storage_path}")

In [3]:
# Set up experiment name
mlflow.set_experiment("XGB_Models")  

<Experiment: artifact_location='file:///Users/lukasfichtner/Documents/Guayas_project_week3/mlflow_results/516751764258600196', creation_time=1757056613364, experiment_id='516751764258600196', last_update_time=1757056613364, lifecycle_stage='active', name='XGB_Models', tags={}>

In [4]:
from pyngrok import ngrok, conf
import getpass
import subprocess
import os

In [5]:
# First, kill any existing ngrok processes
subprocess.run(["pkill", "-f", "ngrok"])
subprocess.run(["pkill", "-9", "-f", "ngrok"])  # Force kill if needed

print("Existing ngrok processes terminated")

Existing ngrok processes terminated


In [6]:
# Launch MLflow UI on port 5000
subprocess.Popen(["mlflow", "ui", "--backend-store-uri", mlflow_storage_path]) 

<Popen: returncode: None args: ['mlflow', 'ui', '--backend-store-uri', '/Use...>

INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)
INFO:     Started parent process [4615]
INFO:     Started server process [4619]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [4617]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [4620]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [4618]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


INFO:     127.0.0.1:52373 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:52373 - "GET /ajax-api/2.0/mlflow/experiments/search?max_results=25&order_by=last_update_time+DESC HTTP/1.1" 200 OK


In [30]:
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = input()
port=5000
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel (ULR for MLFLow UI) \"{public_url}\" ')

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
 * ngrok tunnel (ULR for MLFLow UI) "https://673dbe1b25f5.ngrok-free.app" 


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from darts import TimeSeries
import requests
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance, plot_tree
from xgboost.sklearn import XGBModel 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score , mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV 


In [8]:
df_boost = pd.read_pickle("/Users/lukasfichtner/Documents/Guayas_project_week3/xgboost_dataset.pkl")


In [9]:

#Splitting the train and test data by a specified date
split_date = '2014-01-01'
train = df_boost[df_boost['date'] < split_date]
test = df_boost[df_boost['date'] > split_date]

INFO:     127.0.0.1:52375 - "GET /ajax-api/2.0/mlflow/experiments/search?max_results=25&order_by=last_update_time+DESC HTTP/1.1" 200 OK
INFO:     127.0.0.1:52375 - "GET /ajax-api/2.0/mlflow/experiments/get?experiment_id=516751764258600196 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52393 - "POST /ajax-api/2.0/mlflow/experiments/search-datasets HTTP/1.1" 200 OK
INFO:     127.0.0.1:52393 - "GET /ajax-api/2.0/mlflow/gateway-proxy?gateway_path=api%2F2.0%2Fendpoints%2F HTTP/1.1" 200 OK
INFO:     127.0.0.1:52394 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:52375 - "POST /ajax-api/2.0/mlflow/logged-models/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:52375 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=6cd029271e894ee8aac3df5a3e044906 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52375 - "GET /ajax-api/2.0/mlflow/model-versions/search?filter=tags.%60mlflow.prompt.is_prompt%60+%3D+%27true%27+AND+tags.%60mlflow.prompt.run_ids%60+ILIKE+%22%25d8393e4dd490434b9da0789ec53a8b50%25%22 HTT

In [10]:
# Define target variable (unit_sales) and features
X_train = train.drop('unit_sales', axis=1)
X_train = X_train.drop('date', axis=1)
y_train = train['unit_sales']

X_test = test.drop('unit_sales', axis=1)
X_test = X_test.drop('date', axis=1)
y_test = test['unit_sales']

In [11]:
# Ensure features are scaled for XGBoost (optional, but can help with convergence)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Initialize the XGBoost regressor
xgboost_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=5,
    eta=0.1,
    enable_categorical=True
)

xgboost_model.fit(X_train, y_train) 

# Make predictions on the test set
y_pred = xgboost_model.predict(X_test)

INFO:     127.0.0.1:52400 - "GET /ajax-api/2.0/mlflow/model-versions/search?filter=tags.%60mlflow.prompt.is_prompt%60+%3D+%27true%27+AND+tags.%60mlflow.prompt.run_ids%60+ILIKE+%22%25d8393e4dd490434b9da0789ec53a8b50%25%22 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52400 - "POST /ajax-api/2.0/mlflow/experiments/search-datasets HTTP/1.1" 200 OK
INFO:     127.0.0.1:52400 - "GET /ajax-api/2.0/mlflow/gateway-proxy?gateway_path=api%2F2.0%2Fendpoints%2F HTTP/1.1" 200 OK
INFO:     127.0.0.1:52401 - "POST /ajax-api/2.0/mlflow/logged-models/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:52402 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:52402 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=6cd029271e894ee8aac3df5a3e044906 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52404 - "POST /ajax-api/2.0/mlflow/runs/delete HTTP/1.1" 200 OK
INFO:     127.0.0.1:52406 - "POST /ajax-api/2.0/mlflow/runs/delete HTTP/1.1" 200 OK
INFO:     127.0.0.1:52408 - "POST /ajax-api/2.0/mlflow/runs/delete HTTP/1.

In [13]:
def forecast_metrics(y_true, y_pred):
    #converts input data into flattened NumPy arrays
    y_true = np.asarray(y_true, dtype=float).flatten()
    y_pred = np.asarray(y_pred, dtype=float).flatten()

    # basic errors
    errors       = y_true - y_pred
    abs_errors   = np.abs(errors)
    pct_errors   = abs_errors / np.where(y_true == 0, np.nan, y_true)   # avoid ÷0

    # core statistics
    mae   = abs_errors.mean()
    bias  = errors.mean()
    rmse  = np.sqrt((errors ** 2).mean())

    # MAD of the ACTUAL series (dispersion of demand itself)
    mad   = np.abs(y_true - y_true.mean()).mean()

    # Relative MAD = MAE divided by MAD  (how big the forecast error is vs. “typical” variation)
    rmad  = mae / mad if mad else np.nan

    # Mean Absolute Percentage Error
    mape  = np.nanmean(pct_errors) * 100   # expressed in %

    return {
        "MAE" : mae,
        "Bias": bias,
        "MAD" : mad,
        "rMAD": rmad,
        "MAPE": mape,
        "RMSE": rmse
    }

metrics = forecast_metrics(y_test, y_pred)
for k, v in metrics.items():
    print(f"{k}: {v:,.2f}")

MAE: 0.90
Bias: -0.21
MAD: 5.29
rMAD: 0.17
MAPE: 31.89
RMSE: 4.90


In [14]:
def save_forecast_plot(y_test, y_pred, filename='forecast_plot.png', title='Actual vs Predicted'):
    try:
        # Convert inputs to numpy arrays if they are pandas Series
        y_test = np.array(y_test)
        y_pred = np.array(y_pred)
        
        # Create index for x-axis (e.g., sample index or time steps)
        x_axis = np.arange(len(y_test))
        
        # Create the plot
        plt.figure(figsize=(12, 6))
        
        # Plot actual values
        plt.plot(x_axis, y_test, label='Actual', color='blue', alpha=0.7, linewidth=2)
        
        # Plot predicted values
        plt.plot(x_axis, y_pred, label='Predicted', color='red', alpha=0.7, linewidth=2, linestyle='--')
        
        # Calculate metrics for the plot
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Add metrics to plot
        metrics_text = f'MSE: {mse:.4f}\nMAE: {mae:.4f}\nR²: {r2:.4f}'
        plt.text(0.02, 0.98, metrics_text, transform=plt.gca().transAxes,
                 verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        # Add labels and title
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.title(title)
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Save the plot with proper error handling
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"Plot successfully saved as {filename}")
        return True
        
    except Exception as e:
        print(f"Error saving plot {filename}: {e}")
        plt.close()  # Ensure figure is closed even if error occurs
        return False

In [15]:
with mlflow.start_run(run_name="xgb-normal"):
    # Train model
    xgboost_model.fit(X_train, y_train)
    y_pred = xgboost_model.predict(X_test)
    metrics = forecast_metrics(y_test, y_pred)
    
    # Log all parameters (not just a subset)
    model_params = xgboost_model.get_params()
    for param_name, param_value in model_params.items():
        mlflow.log_param(param_name, param_value)
    
    # Log metrics
    for metric_name, metric_value in metrics.items():
        mlflow.log_metric(metric_name, metric_value)
    
    # Log dataset information
    mlflow.log_param("train_size", len(X_train))
    mlflow.log_param("test_size", len(X_test))
    mlflow.log_param("feature_count", X_train.shape[1])
    mlflow.log_param("split_date", split_date)
    
    # Log feature names
    mlflow.log_param("features", list(X_train.columns))
    
    # Log model with signature and input example
    signature = mlflow.models.infer_signature(X_train, y_pred)
    input_example = X_train.iloc[:5]
    
    mlflow.xgboost.log_model(
        xgboost_model, 
        artifact_path="model",
        signature=signature,
        input_example=input_example
    )
    
    # Log feature importance plot
    plt.figure(figsize=(10, 8))
    plot_importance(xgboost_model, max_num_features=20)
    plt.title("Feature Importance")
    plt.tight_layout()
    mlflow.log_figure(plt.gcf(), "feature_importance.png")
    plt.close()
    
    # Log forecast plot
    save_forecast_plot(y_test, y_pred, "forecast_plot.png", "XGBoost Forecast vs Actual")
    mlflow.log_artifact("forecast_plot.png")
    
    # Set tags for better organization
    mlflow.set_tag("model_type", "xgboost")
    mlflow.set_tag("training_type", "baseline")
    mlflow.set_tag("dataset", "guayas_sales")

INFO:     127.0.0.1:52413 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK


/Users/lukasfichtner/Documents/Guayas_project_week3/.venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/09/05 10:47:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/lukasfichtner/Documents/Guayas_project_week3/.venv/lib/python3.13/site-packages/xgboost

Plot successfully saved as forecast_plot.png


<Figure size 1000x800 with 0 Axes>

INFO:     127.0.0.1:52421 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK


In [25]:
def enhanced_hyperparameter_tuning():
    # Define parameter grid
    param_grid = {
    'eta': [0.01, 0.05, 0.1, 0.3], # Extend learning rates
    'max_depth': [ 3, 5, 6, 7, 9], # Wider depth range
    'subsample': [ 0.7, 0.8, 0.9, 1.0], # More subsampling options
    'colsample_bytree': [ 0.7, 0.8, 0.9, 1.0], # More features samplig
    'n_estimators': [50, 100, 200], # Wider tree count range
    'min_child_weight': [1, 3, 5, 7], # better Control of overitting
    'gamma': [0, 0.1, 0.2, 0.3, 0.4], # Minimum loss reduction
    'lambda': [0.5, 1, 1.5, 2], # Regularization
    'alpha': [0, 0.1, 0.5, 1] # Regularization
    }
    
    # Create parent run for hyperparameter tuning
    with mlflow.start_run(run_name="xgb-hyperparameter-tuning") as parent_run:
        mlflow.set_tag("model_type", "xgboost")
        mlflow.set_tag("training_type", "hyperparameter_tuning")
        
        # TimeSeries cross-validation
        tscv = TimeSeriesSplit(n_splits=5)
        
        # RandomizedSearchCV
        random_search = RandomizedSearchCV(
            estimator=xgb.XGBRegressor(objective='reg:squarederror', random_state=42),
            param_distributions=param_grid,
            n_iter=20,  
            scoring='neg_mean_absolute_error',
            cv=tscv,
            verbose=2,
            n_jobs=-1,
            random_state=42
        )
        
        # Fit the random search
        random_search.fit(X_train, y_train)
        
        # Log overall tuning results
        mlflow.log_params({"n_iter": 50, "cv_splits": 5})
        mlflow.log_metric("best_cv_score", -random_search.best_score_)
        
        # Get best model
        best_model = random_search.best_estimator_
        best_params = random_search.best_params_
        
        # Log best parameters
        for param_name, param_value in best_params.items():
            mlflow.log_param(f"best_{param_name}", param_value)
        
        # Evaluate on test set
        y_pred_tuned = best_model.predict(X_test)
        tuned_metrics = forecast_metrics(y_test, y_pred_tuned)
        # Log forecast plot with proper error handling
        plot_filename = "best_model_forecast.png"
        plot_saved = save_forecast_plot(y_test, y_pred_tuned, plot_filename, "Best Tuned Model Forecast")
    
        if plot_saved and os.path.exists(plot_filename):
            mlflow.log_artifact(plot_filename)
            print("Forecast plot logged to MLflow")
        else:
            print("Warning: Forecast plot could not be created or found")


        
        # Create nested run for the best model
        with mlflow.start_run(run_name="xgb-best-tuned-model", nested=True) as best_run:
            # Log all best parameters
            for param_name, param_value in best_params.items():
                mlflow.log_param(param_name, param_value)
            
            # Log metrics
            for metric_name, metric_value in tuned_metrics.items():
                mlflow.log_metric(metric_name, metric_value)
            
            # Log model with signature
            signature = mlflow.models.infer_signature(X_train, y_pred_tuned)
            input_example = X_train.iloc[:5]
            
            mlflow.xgboost.log_model(
                best_model,
                artifact_path="best_model",
                signature=signature,
                input_example=input_example,
                registered_model_name="XGBoost_Sales_Forecast"
            )
            
            # Log feature importance
            plt.figure(figsize=(10, 8))
            plot_importance(best_model, max_num_features=20)
            plt.title("Best Model Feature Importance")
            plt.tight_layout()
            mlflow.log_figure(plt.gcf(), "best_model_feature_importance.png")
            plt.close()
            
            # Log forecast plot
            save_forecast_plot(y_test, y_pred_tuned, "best_model_forecast.png", "Best Tuned Model Forecast")
            mlflow.log_artifact("best_model_forecast.png")
    
        # Log all CV results as artifact
        cv_results_df = pd.DataFrame(random_search.cv_results_)
        cv_results_df.to_csv("cv_results.csv", index=False)
        mlflow.log_artifact("cv_results.csv")
        
        return best_model, best_params, tuned_metrics

# Run enhanced hyperparameter tuning
best_model, best_params, tuned_metrics = enhanced_hyperparameter_tuning()

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END alpha=0.1, colsample_bytree=0.9, eta=0.05, gamma=0.2, lambda=0.5, max_depth=3, min_child_weight=7, n_estimators=50, subsample=0.9; total time=   2.4s
[CV] END alpha=0.1, colsample_bytree=0.9, eta=0.05, gamma=0.2, lambda=0.5, max_depth=3, min_child_weight=7, n_estimators=50, subsample=0.9; total time=   3.9s
[CV] END alpha=0.1, colsample_bytree=0.9, eta=0.05, gamma=0.2, lambda=0.5, max_depth=3, min_child_weight=7, n_estimators=50, subsample=0.9; total time=   5.5s
[CV] END alpha=0.1, colsample_bytree=0.9, eta=0.05, gamma=0.2, lambda=0.5, max_depth=3, min_child_weight=7, n_estimators=50, subsample=0.9; total time=   6.7s
[CV] END alpha=0.1, colsample_bytree=0.9, eta=0.05, gamma=0.2, lambda=0.5, max_depth=3, min_child_weight=7, n_estimators=50, subsample=0.9; total time=   8.6s
[CV] END alpha=0.1, colsample_bytree=0.9, eta=0.3, gamma=0.2, lambda=1, max_depth=7, min_child_weight=5, n_estimators=100, subsample=0.7; total

/Users/lukasfichtner/Documents/Guayas_project_week3/.venv/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


INFO:     127.0.0.1:52856 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:52856 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=568bfe0e1fec4d0e904c76f32f120848 HTTP/1.1" 200 OK
[CV] END alpha=0.5, colsample_bytree=0.9, eta=0.05, gamma=0.2, lambda=1, max_depth=5, min_child_weight=5, n_estimators=200, subsample=0.8; total time=  12.7s
[CV] END alpha=1, colsample_bytree=0.8, eta=0.01, gamma=0.3, lambda=0.5, max_depth=9, min_child_weight=7, n_estimators=50, subsample=0.8; total time=  16.5s
[CV] END alpha=0.5, colsample_bytree=0.9, eta=0.01, gamma=0.2, lambda=0.5, max_depth=6, min_child_weight=1, n_estimators=200, subsample=1.0; total time=  26.8s
[CV] END alpha=1, colsample_bytree=0.8, eta=0.3, gamma=0.3, lambda=1.5, max_depth=5, min_child_weight=7, n_estimators=200, subsample=1.0; total time=   6.5s
[CV] END alpha=0.5, colsample_bytree=0.9, eta=0.01, gamma=0.2, lambda=0.5, max_depth=6, min_child_weight=1, n_estimators=200, subsample=1.0; total time=  33.6s
[CV]

/Users/lukasfichtner/Documents/Guayas_project_week3/.venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/09/05 12:27:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/lukasfichtner/Documents/Guayas_project_week3/.venv/lib/python3.13/site-packages/xgboost

Plot successfully saved as best_model_forecast.png


<Figure size 1000x800 with 0 Axes>

INFO:     127.0.0.1:52868 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:52868 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=568bfe0e1fec4d0e904c76f32f120848 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52870 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:52870 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=568bfe0e1fec4d0e904c76f32f120848 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52871 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:52871 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=568bfe0e1fec4d0e904c76f32f120848 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52872 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:52872 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=568bfe0e1fec4d0e904c76f32f120848 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52874 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:52874 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=568bfe0e1fec4d0e904c76f32f1208

In [26]:
def enhanced_log_candidate(run_name, params, y_true, y_pred, model, X_train):
    with mlflow.start_run(run_name=run_name, nested=True):
        # Log all parameters
        mlflow.log_params(params)
        
        # Log metrics
        metrics = forecast_metrics(y_true, y_pred)
        for metric_name, metric_value in metrics.items():
            mlflow.log_metric(metric_name, metric_value)
        
        # Log model with signature
        signature = mlflow.models.infer_signature(X_train, y_pred[:5])  # Small sample for example
        input_example = X_train.iloc[:2]
        
        mlflow.xgboost.log_model(
            model,
            artifact_path="model",
            signature=signature,
            input_example=input_example
        )
        
        # Log forecast plot
        plot_path = f"{run_name}_forecast.png"
        save_forecast_plot(y_true, y_pred, plot_path, f"{run_name} Forecast")
        mlflow.log_artifact(plot_path)
        
        # Log feature importance if available
        try:
            plt.figure(figsize=(10, 6))
            plot_importance(model, max_num_features=15)
            plt.title(f"{run_name} Feature Importance")
            plt.tight_layout()
            importance_path = f"{run_name}_feature_importance.png"
            plt.savefig(importance_path)
            mlflow.log_artifact(importance_path)
            plt.close()
        except Exception as e:
            print(f"Could not create feature importance plot: {e}")

In [27]:
def log_data_summary():
    """Log summary statistics about the dataset"""
    with mlflow.start_run(run_name="data-summary", nested=True):
        # Log dataset statistics
        mlflow.log_param("total_samples", len(df_boost))
        mlflow.log_param("train_samples", len(X_train))
        mlflow.log_param("test_samples", len(X_test))
        mlflow.log_param("date_range", f"{df_boost['date'].min()} to {df_boost['date'].max()}")
        
        # Log target variable statistics
        mlflow.log_metric("target_mean", y_train.mean())
        mlflow.log_metric("target_std", y_train.std())
        mlflow.log_metric("target_max", y_train.max())
        mlflow.log_metric("target_min", y_train.min())
        
        # Log missing values info
        missing_info = X_train.isnull().sum()
        mlflow.log_dict(missing_info.to_dict(), "missing_values.json")

INFO:     127.0.0.1:53026 - "GET /ajax-api/2.0/mlflow/model-versions/search?filter=run_id%3D%27a22323a3a90f4687b0f2f82669b1f429%27 HTTP/1.1" 200 OK
INFO:     127.0.0.1:53025 - "GET /ajax-api/2.0/mlflow/model-versions/search?filter=tags.%60mlflow.prompt.is_prompt%60+%3D+%27true%27+AND+tags.%60mlflow.prompt.run_ids%60+ILIKE+%22%25a22323a3a90f4687b0f2f82669b1f429%25%22 HTTP/1.1" 200 OK
INFO:     127.0.0.1:53030 - "GET /static-files/static/js/6791.64608625.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:53031 - "GET /static-files/static/js/5009.73d4294a.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:53029 - "GET /static-files/static/css/5009.e39511b7.chunk.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:53031 - "GET /ajax-api/2.0/mlflow/model-versions/get?name=XGBoost_Sales_Forecast&version=7 HTTP/1.1" 200 OK
INFO:     127.0.0.1:53030 - "GET /model-versions/get-artifact?path=MLmodel&name=XGBoost_Sales_Forecast&version=7 HTTP/1.1" 200 OK
INFO:     127.0.0.1:53031 - "GET /ajax-api/2.0/mlflow/runs/get?

In [ ]:
"""# Log data summary first
log_data_summary()

# Run baseline model
with mlflow.start_run(run_name="xgb-baseline"):
    # Train model
    xgboost_model.fit(X_train, y_train)
    y_pred = xgboost_model.predict(X_test)
    metrics = forecast_metrics(y_test, y_pred)
    
    # Log all parameters
    model_params = xgboost_model.get_params()
    for param_name, param_value in model_params.items():
        mlflow.log_param(param_name, param_value)
    
    # Log metrics
    for metric_name, metric_value in metrics.items():
        mlflow.log_metric(metric_name, metric_value)
    
    # Log dataset information
    mlflow.log_param("train_size", len(X_train))
    mlflow.log_param("test_size", len(X_test))
    mlflow.log_param("feature_count", X_train.shape[1])
    mlflow.log_param("split_date", split_date)
    
    # Log feature names
    mlflow.log_param("features", list(X_train.columns))
    
    # Log model with signature and input example
    signature = mlflow.models.infer_signature(X_train, y_pred)
    input_example = X_train.iloc[:5]
    
    mlflow.xgboost.log_model(
        xgboost_model, 
        artifact_path="model",
        signature=signature,
        input_example=input_example
    )
    
    # Log feature importance plot
    plt.figure(figsize=(10, 8))
    plot_importance(xgboost_model, max_num_features=20)
    plt.title("Feature Importance")
    plt.tight_layout()
    mlflow.log_figure(plt.gcf(), "feature_importance.png")
    plt.close()
    
    # Log forecast plot using mlflow.log_figure to avoid file issues
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.values, label='Actual', color='blue', alpha=0.7, linewidth=2)
    plt.plot(y_pred, label='Predicted', color='red', alpha=0.7, linewidth=2, linestyle='--')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.title('XGBoost Forecast vs Actual')
    plt.legend()
    plt.grid(True, alpha=0.3)
    mlflow.log_figure(plt.gcf(), "forecast_plot.png")
    plt.close()
    
    # Set tags for better organization
    mlflow.set_tag("model_type", "xgboost")
    mlflow.set_tag("training_type", "baseline")
    mlflow.set_tag("dataset", "guayas_sales")

# Run hyperparameter tuning
best_model, best_params, tuned_metrics = enhanced_hyperparameter_tuning()

# Compare results and log final summary
with mlflow.start_run(run_name="final-comparison"):
    baseline_metrics = forecast_metrics(y_test, y_pred)
    
    mlflow.log_metrics({
        "baseline_mae": baseline_metrics["MAE"],
        "tuned_mae": tuned_metrics["MAE"],
        "improvement_mae": baseline_metrics["MAE"] - tuned_metrics["MAE"],
        "improvement_mae_pct": ((baseline_metrics["MAE"] - tuned_metrics["MAE"]) / baseline_metrics["MAE"]) * 100
    })
    
    # Log which model performed better
    if tuned_metrics["MAE"] < baseline_metrics["MAE"]:
        mlflow.set_tag("best_model", "tuned")
    else:
        mlflow.set_tag("best_model", "baseline")"""